<a href="https://colab.research.google.com/github/PaoloBPP/Projeto_Avalia-o_Localizador/blob/main/Localizador_Pessoas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [110]:
#Configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('chave_secreta')
genai.configure(api_key=api_key)

In [3]:
#Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [4]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [5]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [6]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [53]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Este código serve como um filtro buscar algo que queira como por exemplo se a pessoa foi cadastrada
Um exemplo de pesquisa
(Encontrado, Desaparecido, Alojado)

### eqwwqeewq

In [141]:
chat = model.start_chat(history=[])

while prompt != "fim":
    # Input fields
    nome = input("Digite seu nome  (ou 'fim' para parar): ")
    if nome == "fim":
        break

    idade = int(input("Digite sua idade: "))
    cpf = input("Digite seu CPF: ")
    endereco = input("Digite seu endereço: ")
    telefone = input("Digite seu telefone: ")
     # Gender selection using menu
    while True:
        print("\nSelecione seu gênero:")
        print("1. Masculino")
        print("2. Feminino")
        print("3. Infantil")

        escolha_genero = input("Digite sua escolha (1, 2 ou 3): ")

        if escolha_genero in ["1", "2", "3"]:
            if escolha_genero == "1":
                genero = "Masculino"
            elif escolha_genero == "2":
                genero = "Feminino"
            else:
                genero = "Infantil"
            break
        else:
            print("Opção inválida. Digite 1, 2 ou 3.")
     # Gender selection using menu
    while True:
        print("\nSelecione seu status:")
        print("1. Encontrado")
        print("2. Alojado")
        print("3. Desaparecido")

        escolha_status = input("Digite sua escolha (1, 2 ou 3): ")

        if escolha_status in ["1", "2", "3"]:
            if escolha_status == "1":
                status = "Encontrado"
            elif escolha_status == "2":
                status = "Alojado"
            else:
                status = "Desaparecido"
            break
        else:
            print("Opção inválida. Digite 1, 2 ou 3.")




    # Displaying user input
    print("Nome:", nome)
    print("Idade:", idade)
    print("CPF:", cpf)
    print("Telefone:", telefone)
    print("Endereço:", endereco)
    print("Sexo:", genero)
    print("Sttus:", status)

    # Constructing a prompt with user inputs
    prompt = f"Olá, meu nome é {nome}, tenho {idade} anos, meu CPF é {cpf}, meu Numero telefome é {telefone}, moro em {endereco}, sou {genero} e estou {status}."

    # Sending prompt to the model
    response = chat.send_message(prompt)


Digite seu nome  (ou 'fim' para parar): paulo
Digite sua idade: 23
Digite seu CPF: 132
Digite seu endereço: 321
Digite seu telefone: 312

Selecione seu gênero:
1. Masculino
2. Feminino
3. Infantil
Digite sua escolha (1, 2 ou 3): 54
Opção inválida. Digite 1, 2 ou 3.

Selecione seu gênero:
1. Masculino
2. Feminino
3. Infantil
Digite sua escolha (1, 2 ou 3): 1

Selecione seu status:
1. Encontrado
2. Alojado
3. Desaparecido
Digite sua escolha (1, 2 ou 3): 1
Nome: paulo
Idade: 23
CPF: 132
Telefone: 312
Endereço: 321
Sexo: Masculino
Sttus: Encontrado
Digite seu nome  (ou 'fim' para parar): ricardo
Digite sua idade: 32
Digite seu CPF: 213
Digite seu endereço: 312
Digite seu telefone: 312

Selecione seu gênero:
1. Masculino
2. Feminino
3. Infantil
Digite sua escolha (1, 2 ou 3): 1

Selecione seu status:
1. Encontrado
2. Alojado
3. Desaparecido
Digite sua escolha (1, 2 ou 3): 1
Nome: ricardo
Idade: 32
CPF: 213
Telefone: 312
Endereço: 312
Sexo: Masculino
Sttus: Encontrado
Digite seu nome  (ou 'f

In [142]:
#Melhorando a visualização
#Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Imprimindo o histórico
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-------------------------------------------')

> **user**: Olá, meu nome é paulo, tenho 23 anos, meu CPF é 132, meu Numero telefome é 312, moro em 321, sou Masculino e estou Encontrado.

-------------------------------------------


> **model**: Olá, Paulo. Aqui estão as informações que você forneceu:
> 
> * Nome: Paulo
> * Idade: 23 anos
> * CPF: 132
> * Número de telefone: 312
> * Endereço: 321
> * Sexo: Masculino
> * Status de relacionamento: Encontrado

-------------------------------------------


> **user**: Olá, meu nome é ricardo, tenho 32 anos, meu CPF é 213, meu Numero telefome é 312, moro em 312, sou Masculino e estou Encontrado.

...

-------------------------------------------


> **user**: Olá, meu nome é joao, tenho 14 anos, meu CPF é 321, meu Numero telefome é 312, moro em 312, sou Infantil e estou Desaparecido.

-------------------------------------------


> **model**: Olá, João. Sinto muito em saber que você está desaparecido. Aqui estão as informações que você forneceu:
> 
> * Nome: João
> * Idade: 14 anos
> * CPF: 321
> * Número de telefone: 312
> * Endereço: 312
> * Sexo: Infantil
> * Status de relacionamento: Desaparecido
> 
> Por favor, forneça mais informações sobre seu desaparecimento para que possamos ajudá-lo a retornar para casa com segurança.

-------------------------------------------


In [144]:
# ... (import statements and to_markdown function)

# Enhanced filter_history function (replace with your custom logic)
def filter_history(chat_history, search_term):
  # Implement logic to handle complex search queries using regular expressions,
  # custom search logic, or other appropriate methods.
  # Return a list of messages that match the search criteria.
  filtered_messages = []
  # ... (your filtering logic here)
  return filtered_messages

# ... (print_history function)

# Example usage with search for "registered"
search_term = input("Digite um termo de pesquisa (ou deixe em branco para mostrar todas as mensagens): ")

if search_term.lower() == "registrado":
  # Example of a more specific search using custom logic
  # (replace with your actual logic to check for registration)
  print("Verificando se alguém foi registrado...")
  registered_users = find_registered_users(chat_history)  # Hypothetical function
  if registered_users:
    print(f"Usuários registrados encontrados:")
    for user in registered_users:
      print(user)  # Display user information
  else:
    print("Nenhum usuário registrado encontrado.")
else:
  # Regular search using filter_history
  print_history(chat_history, search_term)

Digite um termo de pesquisa (ou deixe em branco para mostrar todas as mensagens): Desaparecido


NameError: name 'chat_history' is not defined